In [9]:
import numpy as np
import cv2
import requests
import json


In [10]:

# Camera Init
cap = cv2.VideoCapture(0) 
cap.set(cv2.CAP_PROP_FPS, 30)

True

In [11]:

# Your PixLab API key
key = '0b55e935a17d52f0a14b7ed4f94f45a4'

# If set to True then composite the flower crown. Otherwise, composite the dog stuff.
draw_crown = False

# Resize an image (Dog parts or the flower crown) to fit the face dimension using smartresize.
def smart_resize(img,width,height):
    print ("Resizing filter image...")
    req = requests.get('https://api.pixlab.io/smartresize',params={
		'img':img,
		'key':key,
		'width': width,
		'height': height
	})
    reply = req.json()
    if reply['status'] != 200:
        print (reply['error'])
        exit()
    else:
        return reply['link'] # Resized image

In [12]:
# The SAP crown.
#flower = 'http://pixlab.xyz/images/flower_crown.png'
crown='Crown.png'
badge='Badge.png'


In [16]:
while 1:
    ret, img = cap.read()
    req = requests.get('https://api.pixlab.io/facelandmarks',params={
        'img': img,'key': key})
    reply = req.json()
    if reply['status'] != 200:
        print (reply['error'])
        exit();
        
    total = len(reply['faces']) # Total detected faces
    if total < 1:
        print ("No faces were detected..exiting")
        exit()
    print(str(total)+" faces were detected")

    # This list contain all the coordinates of the regions where the flower crown or the dog facial parts should be
    # Composited on top of the target face later using the `merge` command.
    coordinates = []

    # Iterate all over the detected faces and make our stuff
    for face in reply['faces']:
        cord = face['rectangle']
        landmarks = face['landmarks']
        fit_crown = smart_resize(
            crown,
            cord['width'] + 20, 
            0 )
        print ("\tCrown flower at: X: " + str(landmarks['bone']['center']['x']) + ", Y: "+str(landmarks['bone']['center']['y']))
        coordinates.append({
            'img': fit_crown, # The resized crown flower
		   'x': landmarks['bone']['center']['x'],
		   'y': landmarks['bone']['center']['y'] + 5,
		   'center':   True,
		   'center_y': True
		})
	


        # Finally, Perform the composite operation
        print ("Composite operation...")
        req = requests.post('https://api.pixlab.io/merge',
	    headers={'Content-Type':'application/json'},
	    data=json.dumps({
            'src':img, # The target image
            'key':key,
            'cord': coordinates }))
        reply = req.json()
        if reply['status'] != 200:
            print (reply['error'])
        else:
            print ("Snap Filter Effect: "+ reply['link'])
        
        cv2.imshow('Thug Life',reply['link'])
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

        k = cv2.waitKey(30) & 0xff
        if k == 27:
            cv2.imwrite('img.jpg', img)
            break

    


TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-16-d8fc1c0dd8e3>, line 29)

In [ ]:

cap.release()
cv2.destroyAllWindows()